In [56]:
import numpy as np # linear algebra
import pandas as pd # data processing
import seaborn as sns 
import matplotlib.pyplot as plt
import missingno
from collections import  Counter

In [57]:
# concatenating the training and the testing data so we have more values to work with and therefor we can better approximate the missing values
# the combined data differentiates the training set and the test set by encoding them with 1 and 0 respectively
training = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
training['train_test'] = 1
test['train_test'] = 0
test['Survived'] = np.NaN
all_data = pd.concat([training,test])

# matplotlib inline
all_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'train_test'],
      dtype='object')

In [58]:
all_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,train_test
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1


### 1.2 Feature engineering
The data for the cabin and tickes has many classes and is very messy therefore we will try clean it up by leveraging these classes to create new variables that aren't in the training set, Furthermore we will look at if the persons title is related to that persons survival

#### 1.2.3 Title

In [59]:
training.Name.head()
# splits up the data into the persons title 
all_data['title'] = all_data.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())
all_data['title'].value_counts()

Mr              757
Miss            260
Mrs             197
Master           61
Rev               8
Dr                8
Col               4
Mlle              2
Major             2
Ms                2
Lady              1
Sir               1
Mme               1
Don               1
Capt              1
the Countess      1
Jonkheer          1
Dona              1
Name: title, dtype: int64

#### 1.2.1 Grouping Ages

In [60]:
# # grouping ages 
# bins = [0,16,21,26,31,36,41,46,51,56,61,66,71,76,81,86]
# #labels = ['0-15','16-20','21-25','26-30','31-35','36-40','41-45','46-50','51-55','56-60','61-65','71-75','76-80','81-85','86-x']
# labels = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
# all_data['Age'] = pd.cut(all_data['Age'], bins=bins, labels=labels, right=False)
# all_data.head()

#### 1.2.2 Family Size

In [61]:
# feature engineering 
all_data['FamilySize']=all_data['Parch']+all_data['SibSp']+1
all_data=all_data.drop(['SibSp'],axis=1)
all_data=all_data.drop(['Parch'],axis=1)
all_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Cabin,Embarked,train_test,title,FamilySize
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,NaN,S,1,Mr,2
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C85,C,1,Mrs,2
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,STON/O2. 3101282,7.9250,NaN,S,1,Miss,1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,C123,S,1,Mrs,2
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,NaN,S,1,Mr,1


In [62]:
# # Mean of survival by sex

# all_data[['title','Age']].groupby('title', as_index = False).mean().sort_values(by = 'Age', ascending = False)
# # all_data[['Sex', 'Survived']].groupby('Sex', as_index = False).mean().sort_values(by = 'Survived', ascending = False)

In [63]:
# all_data.dtypes
# all_data.groupby(['title'])['Age'].mean()
# all_data.groupby(['title'])['Age'].median()

# Apply fillna function within each group
all_data['Age'] = all_data.groupby(['title']).transform(lambda x: x.fillna(x.mean()))

C:\Users\Raelw\AppData\Local\Temp/ipykernel_36152/28519529.py:6: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the transforming function.
  all_data['Age'] = all_data.groupby(['title']).transform(lambda x: x.fillna(x.mean()))


ValueError: Columns must be same length as key

In [64]:
# all_data = all_data.sort_values(['title'])
# srs = all_data.dropna().groupby(['title']).agg({'Age':'mean'})['Age']
# srs.index = all_data[all_data['Age'].isnull()].index
# all_data['Age'] = all_data['Age'].fillna(value=srs)

ValueError: Length mismatch: Expected axis has 13 elements, new values have 263 elements